In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from get_50_day_moving_average import get_50_day_moving_average

In [2]:
df = pd.read_excel('Open Investment Data 2020-2024.xlsx', '2020-2024 with Splits')


In [20]:
df['Currency (Price / share)'][1] is np.nan

False

In [74]:
df_meta = pd.read_excel('Open Investment Data 2020-2024.xlsx', 'Company MetaData-2024')

In [73]:
df_meta.columns

Index(['Row Number', 'Company Name', 'Ticker Symbol',
       'Date of First (Oldest)\n Purchase',
       'Date of Last (Most Recent) \nPurchase',
       'Total Number of Shares\nPurchased',
       'Total Number of Shares after Split', 'Total Purchase Amount GBP',
       'Total Purchase Amount USD',
       'Average Price per Share GBP\n(=Total Purchase Amount / Total Number of Shares)',
       'Average Price per Share USD\n(=Total Purchase Amount / Total Number of Shares) ',
       'Average Price per Share after Split GBP',
       'Average Price per Share after Split USD',
       'Date of First (Oldest) \nSell', 'Date of Last (Most Recent) \nSell',
       'Total Number of Shares Sold',
       'Total Number of Shares Sold after Split', 'Total Sales Amount GBP',
       'Total Sales Amount USD',
       'Average Sale Price per Share GBP (=Total Sale Amount / Total Number of Shares Sold)\n',
       'Average Sale Price per Share USD (=Total Sale Amount / Total Number of Shares Sold)\n',
     

roi = (current price per share usd - average price per share) / average price per share

In [75]:
df_meta['Average Price per Share USD'] = np.where(df_meta['Average Price per Share after Split USD'].isnull(),
                                                  df_meta['Average Price per Share USD\n(=Total Purchase Amount / Total Number of Shares) '],
                                                  df_meta['Average Price per Share after Split USD'],
)
# replace na with 0
df_meta['Average Price per Share USD'] = df_meta['Average Price per Share USD'].apply(
    lambda x: 0 if pd.isna(x) else x
)
# calculate roi and replace na by 0
df_meta['ROI'] = (df_meta['Current Share Price USD\n(Yahoo & Google Finance)'] - df_meta['Average Price per Share USD']) / df_meta['Average Price per Share USD']
df_meta['ROI'] = df_meta['ROI'].apply(lambda x: 0 if pd.isna(x) else x)

KeyError: 'Current Share Price USD\n (Yahoo & Google  Finance)'

In [72]:
for i in range(len(df_meta)):
    roi = df_meta['ROI'][i]
    if roi == np.inf:
        print(i)

KeyError: 'ROI'

In [51]:
df_meta[['Row Number', 'Ticker Symbol', 'ROI']]

,Row Number,Ticker Symbol,ROI
0,1,MMM,-0.281446
1,2,AOS,0.254255
2,3,ABBV,0.574366
3,4,GOOGL,-0.080099
4,5,AWR,0.037990
...,...,...,...
81,82,WM,0.662279
82,83,WELL,0.900710
83,84,NNN,0.103534
84,85,AMZN,0.025797


In [3]:
df['Price / share adjusted for split(s) (USD)']

0              NaN
1              NaN
2              NaN
3              NaN
4        22.160121
           ...    
3294           NaN
3295           NaN
3296           NaN
3297    212.600000
3298    181.600000
Name: Price / share adjusted for split(s) (USD), Length: 3299, dtype: float64

In [2]:
df = pd.read_csv('2020-2024-split.csv')
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], format='%Y/%m/%d')
# df = df[(df['Action'] == 'Market buy') | (df['Action'] == 'Market sell')]
df_meta = pd.read_csv('company meta 2020-2024.csv')


In [3]:
SNOW = pd.read_csv('stocksprice/SNOW_2020-09-16_2021-02-22_50_day_MA.csv')

In [4]:
def showBuyAndSell(ticker):
    print(df[(df['Action'].isin(['Market buy', 'Market sell']))&(df['Ticker'] == ticker)].groupby('Action')['No. of shares'].sum())

In [7]:
for ticker in ['LVGO', 'DLR', 'NLY', 'MRK', 'AOS', 'NYMT']:
    print(ticker)
    showBuyAndSell(ticker)


LVGO
Action
Market buy    8.5
Name: No. of shares, dtype: float64
DLR
Action
Market buy     101.0
Market sell     37.0
Name: No. of shares, dtype: float64
NLY
Action
Market buy     945.5
Market sell    488.0
Name: No. of shares, dtype: float64
MRK
Action
Market buy     54.0
Market sell     8.0
Name: No. of shares, dtype: float64
AOS
Action
Market buy     87.0
Market sell     6.0
Name: No. of shares, dtype: float64
NYMT
Action
Market buy     1643.5
Market sell     706.0
Name: No. of shares, dtype: float64


In [57]:
df[(df['Action'].isin({'Market buy', 'Market sell'})) & (df['Ticker'] == 'LVGO')]

,Action,Transaction Date,Time,Ticker,Name,No. of shares,Price / share,Price / share adjusted for split(s),Currency (Price / share),Exchange rate,...,Stamp duty reserve tax,Currency (Stamp duty reserve tax),Notes,ID,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
31,Market buy,2020-07-09,13:30:12,LVGO,Livongo,3.5,98.97,NaN,USD,1.2667,...,NaN,NaN,NaN,EOF305052094,NaN,NaN,NaN,NaN,NaN,NaN
33,Market buy,2020-07-31,13:30:16,LVGO,Livongo,5.0,126.57,NaN,USD,1.31455,...,NaN,NaN,NaN,EOF343980549,NaN,NaN,NaN,NaN,NaN,NaN


: 

In [6]:
sell_df = df[df['Action'] == 'Market sell']

In [70]:
dividend_actions = [
        'Dividend (Ordinary)',
        'Dividend (Return of capital non us)',
        'Dividend (Demerger)',
        'Dividend (Bonus)',
        'Dividend (Ordinary manufactured payment)',
        'Dividend (Dividends paid by us corporations)',
        'Dividend (Dividends paid by foreign corporations)',
        'Dividend (Dividend)'
    ]

df[(df['Action'].isin(dividend_actions))&(df['Action'] == 'Dividend (Bonus)')].sort_values('Price / share')

,Action,Transaction Date,Time,Ticker,Name,No. of shares,Price / share,Price / share adjusted for split(s),Currency (Price / share),Exchange rate,...,Stamp duty reserve tax,Currency (Stamp duty reserve tax),Notes,ID,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
976,Dividend (Bonus),2021-12-24,7:39:30,O,Realty Income,40.0,0.00,NaN,USD,Not available,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1731,Dividend (Bonus),2022-12-20,8:30:16,HRZN,Horizon Technology Finance,140.0,0.04,NaN,USD,Not available,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1732,Dividend (Bonus),2022-12-20,8:30:16,HRZN,Horizon Technology Finance,256.0,0.04,NaN,USD,Not available,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
761,Dividend (Bonus),2021-09-07,9:11:47,LTC,LTC Properties,25.0,0.16,NaN,USD,Not available,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
762,Dividend (Bonus),2021-09-07,9:11:47,LTC,LTC Properties,45.0,0.16,NaN,USD,Not available,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1496,Dividend (Bonus),2022-08-05,7:40:58,PSA,Public Storage,4.0,11.18,NaN,USD,Not available,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1499,Dividend (Bonus),2022-08-05,7:40:58,PSA,Public Storage,9.0,11.18,NaN,USD,Not available,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
buy_sell_dividend_df = df[df['Action'].isin(dividend_actions+['Market buy', 'Market sell'])]

In [63]:
buy_sell_dividend_df[buy_sell_dividend_df['Ticker'] == 'STOR']

,Action,Transaction Date,Time,Ticker,Name,No. of shares,Price / share,Price / share adjusted for split(s),Currency (Price / share),Exchange rate,...,Stamp duty reserve tax,Currency (Stamp duty reserve tax),Notes,ID,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
69,Market buy,2020-10-06,16:02:49,STOR,STORE Capital,38.000000,28.95,NaN,USD,1.29438,...,NaN,NaN,NaN,EOF472936374,NaN,NaN,NaN,NaN,NaN,NaN
108,Market buy,2020-11-20,15:57:27,STOR,STORE Capital,10.000000,31.79,NaN,USD,1.32839,...,NaN,NaN,NaN,EOF567740720,NaN,NaN,NaN,NaN,NaN,NaN
120,Market buy,2020-11-27,17:06:20,STOR,STORE Capital,10.000000,33.38,NaN,USD,1.33019,...,NaN,NaN,NaN,EOF610958736,NaN,NaN,NaN,NaN,NaN,NaN
125,Market buy,2020-12-01,18:25:53,STOR,STORE Capital,8.000000,32.71,NaN,USD,1.34197,...,NaN,NaN,NaN,EOF620980708,NaN,NaN,NaN,NaN,NaN,NaN
172,Dividend (Ordinary),2021-01-19,11:12:11,STOR,STORE Capital,66.000000,0.31,NaN,USD,Not available,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
435,Dividend (Ordinary),2021-04-19,16:40:38,STOR,STORE Capital,66.000000,0.31,NaN,USD,Not available,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,Dividend (Ordinary),2021-07-20,12:47:47,STOR,STORE Capital,66.000000,0.31,NaN,USD,Not available,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
727,Market buy,2021-08-25,13:30:01,STOR,STORE Capital,2.000000,35.30,NaN,USD,1.37018,...,NaN,NaN,NaN,NaN,NaN,EOF1512818151,0.08,GBP,NaN,NaN
745,Market buy,2021-08-31,14:26:00,STOR,STORE Capital,3.000000,35.73,NaN,USD,1.37866,...,NaN,NaN,NaN,EOF1522444431,0.12,GBP,NaN,NaN,NaN,NaN
746,Market buy,2021-08-31,14:26:18,STOR,STORE Capital,1.000000,35.70,NaN,USD,1.37851,...,NaN,NaN,NaN,EOF1522444470,0.04,GBP,NaN,NaN,NaN,NaN


In [21]:
df['Action'].unique()

array(['Deposit', 'Market buy', 'Dividend (Ordinary)',
       'Dividend (Return of capital non us)', 'Market sell', 'Withdrawal',
       'Dividend (Demerger)', 'Dividend (Bonus)',
       'Dividend (Ordinary manufactured payment)', 'Company Acquisition',
       nan, 'Dividend (Dividends paid by us corporations)',
       'Interest on cash',
       'Dividend (Dividends paid by foreign corporations)',
       'Dividend (Dividend)'], dtype=object)

In [28]:

from get_50_day_moving_average import get_50_day_moving_average


def getStockPrice(df):
    df_ticker_date = df.groupby('Ticker').agg({
        'Transaction Date': ['min', 'max']
    })
    minDateDict = df_ticker_date['Transaction Date'].to_dict()['min']
    maxDateDict = df_ticker_date['Transaction Date'].to_dict()['max']
    tickerDate = {}
    for ticker in df_ticker_date.index:
        min_date = minDateDict[ticker]
        max_date = maxDateDict[ticker]
        price_data, error = get_50_day_moving_average(
            ticker, min_date.strftime("%Y-%m-%d"), max_date.strftime("%Y-%m-%d"))
        if error:
            print(error)
        else:
            tickerDate[ticker] = {
                'min': min_date,
                'max': max_date,
                'price': price_data
            }

    
    return tickerDate

In [30]:
stockPrice = getStockPrice(df, df_meta)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

No data found for CWEN/A between 2021-09-18 and 2023-11-29


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

No data found for LVGO between 2020-03-31 and 2020-07-31


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['STOR']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


No data found for STOR between 2020-06-28 and 2023-02-06


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['WORK']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


No data found for WORK between 2020-11-09 and 2021-02-18


[*********************100%%**********************]  1 of 1 completed


In [54]:

def addGainAndLossToStockPrice(stockPrice, df_meta):
    # add gain
    for index, row in df_meta.iterrows():
        if row['Ticker Symbol'] in stockPrice.keys():
            stockPrice[row['Ticker Symbol']]['Realized Capital Gain & Loss'] = row['Realized Capital Gain & Loss']

In [78]:
for _, row in df_meta.iterrows():
    print(row['Ticker Symbol'])

MMM
AOS
ABBV
GOOGL
AWR
AMT
NLY
AAPL
T
ADBE
AMAT
BYND
BTBT
AVGO
CAT
CWEN
CWEN/A
NET
CMCSA
COST
CRON
CVS
DLR
ECL
WTRG
EXPE
EXR
FSLY
GOOD
GMRE
HASI
HI
HD
HON
HRZN
ICE
INTU
JNJ
JPM
LVGO
LTC
MTB
MDT
MRK
META
MA
MCO
NG
NFLX
NYMT
NEE
NKE
OHI
ONL
PBCT
PETS
PSA
QCOM
O
RIOT
RITM
CRM
STX
SCI
SBNY
SICP
SPG
WORK
SNOW
SEDG
STOR
SPGI
TGT
UBER
UNP
UNH
VZ
VICI
V
DIS
WBD
WM
WELL


In [55]:
addGainAndLossToStockPrice(stockPrice, df_meta)

In [56]:
stockPrice

{'AAPL': {'min': Timestamp('2020-07-01 00:00:00'),
  'max': Timestamp('2023-11-28 00:00:00'),
  'price':                  Close   50_day_MA
  Date                              
  2020-07-01   91.027496   80.744700
  2020-07-02   91.027496   81.184750
  2020-07-06   93.462502   81.678850
  2020-07-07   93.172501   82.127450
  2020-07-08   95.342499   82.618450
  ...                ...         ...
  2023-11-20  191.449997  177.152601
  2023-11-21  190.639999  177.439401
  2023-11-22  191.309998  177.781401
  2023-11-24  189.970001  178.066001
  2023-11-27  189.789993  178.361601
  
  [858 rows x 2 columns],
  'Realized Capital Gain & Loss': 0.0},
 'ABBV': {'min': Timestamp('2020-11-27 00:00:00'),
  'max': Timestamp('2023-11-28 00:00:00'),
  'price':                  Close   50_day_MA
  Date                              
  2020-11-27  104.889999   90.455999
  2020-11-30  104.580002   90.745399
  2020-12-01  104.040001   91.044400
  2020-12-02  104.989998   91.372200
  2020-12-03  104.7399

In [58]:
colormap = cm.get_cmap('RdYlGn')
def get_color(value, minValue, maxValue, colormap):
    norm = plt.Normalize(vmin=minValue, vmax=maxValue)
    return colormap(norm(value))

C:\Users\84515\AppData\Local\Temp\ipykernel_31292\3310250405.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap('RdYlGn')


In [74]:
tickers = list(stockPrice.keys())
realizedCapitalGainAndLoss = [item['Realized Capital Gain & Loss']for item in stockPrice.values()]
minRealizedCapitalGainAndLoss = min(realizedCapitalGainAndLoss)
maxRealizedCapitalGainAndLoss = max(realizedCapitalGainAndLoss)

for index, ticker in enumerate(tickers):
    data = stockPrice[ticker]
    color = get_color(data['Realized Capital Gain & Loss'], minRealizedCapitalGainAndLoss, maxRealizedCapitalGainAndLoss, colormap)


(np.float64(0.8568242983467899), np.float64(0.9397154940407536), np.float64(0.5530949634755865), np.float64(1.0))
(np.float64(0.8568242983467899), np.float64(0.9397154940407536), np.float64(0.5530949634755865), np.float64(1.0))
(np.float64(0.8568242983467899), np.float64(0.9397154940407536), np.float64(0.5530949634755865), np.float64(1.0))
(np.float64(0.8568242983467899), np.float64(0.9397154940407536), np.float64(0.5530949634755865), np.float64(1.0))
(np.float64(0.827450980392157), np.float64(0.9271049596309113), np.float64(0.5298731257208766), np.float64(1.0))
(np.float64(0.8568242983467899), np.float64(0.9397154940407536), np.float64(0.5530949634755865), np.float64(1.0))
(np.float64(0.8568242983467899), np.float64(0.9397154940407536), np.float64(0.5530949634755865), np.float64(1.0))
(np.float64(0.7725490196078434), np.float64(0.9034217608612073), np.float64(0.49434832756632074), np.float64(1.0))
(np.float64(0.8685121107266437), np.float64(0.9446366782006921), np.float64(0.5690888119

In [76]:
realizedCapitalGainAndLoss = [item['Realized Capital Gain & Loss']for item in stockPrice.values()]
